In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import math

In [3]:
# Input variables
budget = 483.4
gavi = 329.5
paid = 8.7
subs = 65.4
initial = 1.4
contracts = 16.8
new = budget - gavi - paid - subs - initial - contracts

# Sensitivity range for overfill
overfill_values = np.linspace(0, 0.5, 11)  # Testing from 0% to 50% overfill in increments of 5%
sims = 1000000

# Attrition ranges
subs_attrit_low, subs_attrit_high = 0.08, 0.18 #point estimate is 13%
init_attrit_low, init_attrit_high = 0.01, 0.02 #point estimate is 33% but treating differently for early year approvals, need to collapse later
contr_attrit_low, contr_attrit_high = 0.12, 0.22 #point estimate is 17%
new_attrit_low, new_attrit_high = 0.23, 0.43 #point estimate is 33%, need to collapse into initial later
pri_attrit_low, pri_attrit_high = 0.10, 0.75 #point estimate is 65%

# Probability ranges
between_low = budget * 0.97
between_high = budget * 1.03

# Storage for sensitivity analysis
sensitivity_results = []

# Running simulation for different overfill levels
for overfill in overfill_values:
    subs_simulated = (subs / (1 - overfill)) * (1 - np.random.uniform(subs_attrit_low, subs_attrit_high, sims))
    init_simulated = (initial / (1 - overfill)) * (1 - np.random.uniform(init_attrit_low, init_attrit_high, sims))
    contr_simulated = (contracts / (1 - overfill)) * (1 - np.random.uniform(contr_attrit_low, contr_attrit_high, sims))
    new_simulated = (new / (1 - overfill)) * (1 - np.random.uniform(new_attrit_low, new_attrit_high, sims))
    
    payout_simulated = gavi + paid + subs_simulated + init_simulated + contr_simulated + new_simulated
    
    prob_between = np.sum((payout_simulated > between_low) & (payout_simulated < between_high)) / sims
    sensitivity_results.append([overfill, prob_between])

# Convert results to DataFrame
df_sensitivity = pd.DataFrame(sensitivity_results, columns=["Overfill", "Probability Within Budget Range"])

# Display sensitivity results
import ace_tools as tools
tools.display_dataframe_to_user(name="Overfill Sensitivity Analysis", dataframe=df_sensitivity)

# Running one simulation with default overfill (0.30) for visualization
overfill = 0.12
subs_simulated = (subs / (1 - overfill)) * (1 - np.random.uniform(subs_attrit_low, subs_attrit_high, sims))
init_simulated = (initial / (1 - overfill)) * (1 - np.random.uniform(init_attrit_low, init_attrit_high, sims))
contr_simulated = (contracts / (1 - overfill)) * (1 - np.random.uniform(contr_attrit_low, contr_attrit_high, sims))
new_simulated = (new / (1 - overfill)) * (1 - np.random.uniform(new_attrit_low, new_attrit_high, sims))

payout_simulated = gavi + paid + subs_simulated + init_simulated + contr_simulated + new_simulated

# Visualization
fig, ax = plt.subplots(figsize=(10, 6))

# Histogram
sns.histplot(payout_simulated, bins=50, kde=True, color="blue", alpha=0.6, ax=ax)
ax.axvline(budget, color="red", linestyle="--", label="Budget")
ax.axvline(between_low, color="green", linestyle="--", label="97% of Budget")
ax.axvline(between_high, color="green", linestyle="--", label="103% of Budget")

ax.set_title("Distribution of Simulated Payouts")
ax.set_xlabel("Payout Amount ($M)")
ax.set_ylabel("Frequency")
ax.legend()
plt.show()

# CDF Plot
fig, ax = plt.subplots(figsize=(10, 6))
sns.ecdfplot(payout_simulated, color="blue", ax=ax)
ax.axvline(budget, color="red", linestyle="--", label="Budget")
ax.axvline(between_low, color="green", linestyle="--", label="97% of Budget")
ax.axvline(between_high, color="green", linestyle="--", label="103% of Budget")

ax.set_title("Cumulative Distribution Function (CDF) of Payouts")
ax.set_xlabel("Payout Amount ($M)")
ax.set_ylabel("Cumulative Probability")
ax.legend()
plt.show()

ModuleNotFoundError: No module named 'ace_tools'